This is a tutorial for Python 2.7

# pyphysio library

## 2. Pipelines

In this tutorial we consider two pipelines for the processing of ECG and EDA signals respectively.

We divide the pipelines into three separate steps:
1. Filtering and Preprocessing: this step includes all the procedures aiming at increasing the signal/noise ratio, typycally band-pass filtering, smoothing, removal of artifacts. The output of this step is a new version of the imput signal with improved signal quality (less noise);
2. Information Extraction: this step aims at extracting the information of interest from the physiological signal. The output is a new signal containing only the information of interest and thus it has a **signal_nature** different from the input signal.
3. Physiological Indicators: this steps produces a list of scalar values able to describe the characteristics of the input signal. This step is usually performed on small segments of the input signals which are extracted using a sliding window on the whole length of the signal.

![Figure 2: Physiological signal processing flowchart](img/flowchart_processing.png)

*Figure 2: Physiological signal processing flowchart*

To understand how a signal processing step is represented in **pyphysio** see tutorial 1-signals_algorithms

In the following code we will also use the shortened sintax to apply a signal processing step:

```python
# standard sintax: creation + execution
filter_iir = ph.IIRFilter(fp=45, fs = 50, ftype='ellip') # creation of the processing step
ecg_out = filter_iir(ecg) # execution on the input signal


# shortened sintax: creation(execution)
ecg_out = ph.IIRFilter(fp=45, fs = 50, ftype='ellip')(ecg)```

However, the standard sintax is suggested when applying the same processing steps to multiple signals or for the clarity of the scripts:

Es:
```python
filter_iir = ph.IIRFilter(fp=45, fs = 50, ftype='ellip') # creation of the processing step

ecg_1_out = filter_iir(ecg_1) # execution on the input signal
ecg_2_out = filter_iir(ecg_2) # execution on the input signal
ecg_3_out = filter_iir(ecg_3) # execution on the input signal
```

### 2.1 ECG processing pipeline

** Step 0: Import data **

In [1]:
# import libraries
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# for windowed plots
%matplotlib auto 

Using matplotlib backend: TkAgg


In [3]:
# import all pyphysio classes and methods
import pyphysio as ph
ph.__path__

['/home/andrea/Trento/CODICE/workspaces/pyHRV/pyphysio']

In [4]:
# import data and creating a signal

ecg_data = ph.TestData.ecg()

fsamp = 2048
ecg = ph.EvenlySignal(values = ecg_data, sampling_freq = fsamp, signal_nature = 'ecg')

In [ ]:
ecg.plot()

** Step 1: Filtering and preprocessing **

In [5]:
# (optional) IIR filtering : remove high frequency noise
ecg = ph.IIRFilter(fp=45, fs = 50, ftype='ellip')(ecg)

In [6]:
# normalization : normalize data
ecg = ph.Normalize(norm_method='standard')(ecg)

In [7]:
# resampling : increase the sampling frequency by cubic interpolation
ecg = ecg.resample(fout=4096, kind='cubic')
fsamp = 4096

In [ ]:
ecg.plot()

** Step 2: Information Extraction **

The information we want to extract from the ECG signal is the position of the heartbeats and the Inter Beat Interval signal.

In [8]:
ibi = ph.BeatFromECG()(ecg)

In [9]:
ibi.get_duration()

120.0

In [ ]:
# check results so far
ax1 = plt.subplot(211)
ecg.plot()
plt.vlines(ibi.get_times(), np.min(ecg), np.max(ecg))

plt.subplot(212, sharex = ax1)
ibi.plot('o-')
plt.vlines(ibi.get_times(), np.min(ibi), np.max(ibi))
plt.show()

** Step 3: Physiological Indicators **

In [10]:
# define a list of indicators we want to compute
hrv_indicators = [ph.Mean(name='RRmean'), ph.StDev(name='RRstd'), ph.RMSSD(name='rmsSD')]
ph.hrv_indicators

In [11]:
# create fake label
label = np.zeros(1200)

label[300:600] = 1

label[900:1200] = 2

label = ph.EvenlySignal(label, sampling_freq = 10, signal_nature = 'label')

In [12]:
# check label
ax1 = plt.subplot(211)
ibi.plot('.-')

plt.subplot(212, sharex = ax1)
label.plot('.-')
plt.show()

In [14]:
#fixed length windowing
fixed_length = ph.FixedSegments(step = 5, width = 10, labels = label, drop_mixed=True, drop_cut=False)

indicators, col_names = ph.fmap(fixed_length, hrv_indicators, ibi)

In [15]:
indicators[:,0:3]

array([[   0.,   10.,    0.],
       [   5.,   15.,    0.],
       [  10.,   20.,    0.],
       [  15.,   25.,    0.],
       [  20.,   30.,    0.],
       [  25.,   35.,    0.],
       [  30.,   40.,    1.],
       [  35.,   45.,    1.],
       [  40.,   50.,    1.],
       [  45.,   55.,    1.],
       [  50.,   60.,    1.],
       [  55.,   65.,    1.],
       [  60.,   70.,    0.],
       [  65.,   75.,    0.],
       [  70.,   80.,    0.],
       [  75.,   85.,    0.],
       [  80.,   90.,    0.],
       [  85.,   95.,    0.],
       [  90.,  100.,    2.],
       [  95.,  105.,    2.],
       [ 100.,  110.,    2.],
       [ 105.,  115.,    2.],
       [ 110.,  120.,    2.],
       [ 115.,  120.,    2.]])

In [16]:
# extract column with the labels for each window
label_w = indicators[:, np.where(col_names == 'label')[0]]

# extract column with the RRmean values computed from each window
rrmean_w = indicators[:, np.where(col_names == 'RRmean')[0]]

In [17]:
# create a box and whisker plot
plt.boxplot([rrmean_w[np.where(label_w==1)[0]], rrmean_w[np.where(label_w==2)[0]]], labels=['image1', 'image2'])
plt.show()

In [20]:
#label based windowing
label_based = ph.LabelSegments(labels = label, drop_cut=False, drop_mixed=False)

indicators, col_names = ph.fmap(label_based, hrv_indicators, ibi)

indicators[:,0:3]

array([[   0.,   30.,    0.],
       [  30.,   60.,    1.],
       [  60.,   90.,    0.],
       [  90.,  120.,    2.]])

In [19]:
custom_based = ph.CustomSegments(begins = [0, 30, 60, 90], ends = [30, 60, 90, label.get_duration()], labels=label, drop_shorter=False, drop_mixed=False)

indicators, col_names = ph.fmap(custom_based, hrv_indicators, ibi)

indicators[:,0:3]

array([[0, 30, None],
       [30, 60, None],
       [60, 90, None],
       [90, 120.0, None]], dtype=object)

### 2.2 EDA processing pipeline

** Step 0: Import data **

In [ ]:
# import data and creating a signal

eda_data = ph.TestData.eda()

fsamp = 2048
eda = ph.EvenlySignal(values = eda_data, sampling_freq = fsamp, signal_nature = 'eda')

eda.plot()

** Step 1: Filtering and preprocessing **

In [ ]:
# resampling : decrease the sampling frequency by cubic interpolation
eda = eda.resample(fout=8, kind='cubic')

In [ ]:
# IIR filtering : remove high frequency noise
eda = ph.IIRFilter(fp=0.8, fs = 1.1, ftype='ellip')(eda)

In [ ]:
eda.plot()

** Step 2: Information Extraction **

The information we want to extract from the EDA signal is the phasic component associated to the sympathetic activity.

In [ ]:
# estimate the driver function
driver = ph.DriverEstim()(eda)

In [ ]:
# compute the phasic component
phasic, tonic, _ = ph.PhasicEstim(delta=0.02)(driver)
phasic.plot()

In [ ]:
# check results so far
ax1 = plt.subplot(211)
eda.plot()

plt.subplot(212, sharex = ax1)
driver.plot()
phasic.plot()
plt.grid()
plt.show()

** Step 3: Physiological Indicators **

In [ ]:
# define a list of indicators we want to compute
indicators = [ph.Mean(), ph.StDev(), ph.AUC(), 
              ph.PeaksMean(delta=0.02),
              ph.DurationMean(delta=0.02)
             ]


In [ ]:
# define the windowing method
# create fake label
label = np.zeros(1200)

label[300:600] = 1

label[900:1200] = 2

label = ph.EvenlySignal(label, sampling_freq = 10, signal_nature = 'label')

In [ ]:
#fixed length windowing
fixed_length = ph.FixedSegments(step = 5, width = 20, labels = label, drop_mixed=False, drop_shorter=False)

indicators, col_names = ph.fmap(fixed_length(eda), indicators)